# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770833886993         NaN   2.98e-01   0.80    8.0
  2   -2.772142976764   -1.31e-03   4.88e-02   0.80    1.0
  3   -2.772170289122   -2.73e-05   2.83e-03   0.80    1.0
  4   -2.772170697587   -4.08e-07   4.02e-04   0.80    2.0
  5   -2.772170720955   -2.34e-08   1.38e-04   0.80    2.0
  6   -2.772170723014   -2.06e-09   5.88e-06   0.80    2.0
n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770779567920         NaN   2.99e-01   0.80    8.0
  2   -2.772055760128   -1.28e-03   4.78e-02   0.80    1.0
  3   -2.772083000348   -2.72e-05   3.73e-03   0.80    2.0
  4   -2.772083417348   -4.17e-07   5.54e-05   0.80    2.0
  5   -2.772083417792   -4.45e-10   1.01e-05   0.80    2.0


1.773626897043807

## Forward-mode implicit differentiation

Right now DFTK has no out-of-the-box support for implicit differentiation through the SCF.
However one can easily work around this as follows. We keep both a non-dual basis
and a basis including duals for easy bookkeeping (but redundant computation ...).

In [3]:
function self_consistent_field_dual(basis::PlaneWaveBasis, basis_dual::PlaneWaveBasis{T};
                                    kwargs...) where T <: ForwardDiff.Dual
    scfres = self_consistent_field(basis; kwargs...)
    ψ, occupation = DFTK.select_occupied_orbitals(basis, scfres.ψ, scfres.occupation)

    # promote everything eagerly to Dual numbers
    occupation_dual = [T.(occupation[1])]
    ψ_dual = [Complex.(T.(real(ψ[1])), T.(imag(ψ[1])))]
    ρ_dual = compute_density(basis_dual, ψ_dual, occupation_dual)

    _, δH = energy_hamiltonian(basis_dual, ψ_dual, occupation_dual; ρ=ρ_dual)
    δHψ = δH * ψ_dual
    δHψ = [ForwardDiff.partials.(δHψ[1], 1)]
    δψ = DFTK.solve_ΩplusK(basis, ψ, -δHψ, occupation)
    δρ = DFTK.compute_δρ(basis, ψ, δψ, occupation)
    ρ = ForwardDiff.value.(ρ_dual)
    ψ, ρ, δψ, δρ
end;

This function is now used in the following to provide a dual version
for the compute_dipole function:

In [4]:
function compute_dipole(ε::ForwardDiff.Dual; tol=1e-8, kwargs...)
    T = ForwardDiff.tagtype(ε)
    basis = make_basis(ForwardDiff.value(ε); kwargs...)
    basis_dual = make_basis(ε; kwargs...)
    ψ, ρ, δψ, δρ = self_consistent_field_dual(basis, basis_dual; tol)
    ρ_dual = ForwardDiff.Dual{T}.(ρ, δρ)
    dipole(basis_dual, ρ_dual)
end;

This setup allows to compute the polarizability via automatic differentiation:

In [5]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770419711849         NaN   2.94e-01   0.80    8.0
  2   -2.772041476806   -1.62e-03   5.29e-02   0.80    1.0
  3   -2.772083185190   -4.17e-05   1.05e-03   0.80    2.0
  4   -2.772083413814   -2.29e-07   3.12e-04   0.80    3.0
  5   -2.772083417654   -3.84e-09   6.87e-05   0.80    2.0

Polarizability via ForwardDiff:       1.7725772964752893
Polarizability via finite difference: 1.773626897043807
